# Regresiones

Aquí vamos a hacer muchas regresiones para ver qué combinación de ingeniería de variables y método de regresión da mejores resultados.

El primer paso es establecer una _línea base_ para comparar. Para esto vamos a utilizar una regresión de [Random Forest](https://en.wikipedia.org/wiki/Random_forest) sobre los datos preprocesados diréctamente, sin más. Lo único extra que vamos a hacer es escalar las variables.

Hay varias razones para usar Random Forest como baseline, primero es sabido que es un buen método para evitar el _overfitting_ y segundo y más importante en nuestro caso, nos da una estimación de la importancia relativa de las variables, esto puede resultar importante para informar nuestras desiciones más adelante.

En general, una regresión tiene un conjunto de _hiperparámetros_ que deben ajustarse para obtener un modelo específico. Este ajuste se hace buscando soluciones en el espacio de hiperparámetros, para evitar ajustar nuestro modelo demasiado (aumentar el sesgo), vamos a usar _cross validation_.

Primero vamos a ver cuáles son los hiperparámetros que queremos ajustar:

In [17]:
# leemos los datos
import pandas as pd
df_train = pd.read_csv("data/train_preprocesado.csv")
#por alguna razón quedaron dos Na en BsmtCond, así los quitamos pero en realidad hay que regresar a ver qué pasó
df_train.dropna(inplace=True)
# creamos dataframes para las variables y el objetivo
Y = df_train['SalePrice']
X = df_train.drop(['SalePrice','Id'], axis=1)

In [3]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
print('Parámetros por defecto:\n')
pprint(rf.get_params())

Parámetros por defecto:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


- n_estimators = número de árboles en el bosque
- max_features = máximo número de variables a considerar en cada _split_
- max_depth = profundidad máxima para cada árbol
- min_samples_split = numero mínimo de datos en un nodo antes de un _split_
- min_samples_leaf = número mínimo de datos en un nodo
- bootstrap = método para muestrear (con o sin reemplazo)

Si lo piensa uno, son un montón de parámetros con un montón de valores posibles, entonces tenemos que encontrar alguna forma eficiente de buscar (en el espacio de hiperparámetros). Ahora lo que vamos a hacer es una búsqueda en dos etapas. primero vamos a definir una _malla_ grande de valores y vamos a buscar a azar dentro de esa malla, eso nos va a reducir el espacio de búsqueda, luego vamos a usar ese espacio reducido para utilizar un método más sofisticado (pero más costoso computacionalmente) de búsqueda. 

In [5]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Lista ed valores para el número de estimadores
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# máximo número de variables a considerar en cada _split_
max_features = ['auto', 'sqrt']
# profundidad máxima para cada árbol
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# numero mínimo de datos en un nodo antes de un _split_
min_samples_split = [2, 5, 10]
# número mínimo de datos en un nodo
min_samples_leaf = [1, 2, 4]
# método para muestrear (con o sin reemplazo)
bootstrap = [True, False]
# Malla aleatoria
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


Hay exáctamente $2 * 12 * 2 * 3 * 3 * 10 = 4320$ combinaciones de variables en nuestra malla. Por eso no las vamos a probar todas, vamos a buscar aleatoriamente

In [19]:
# Creamos un modelo vacío
rf = RandomForestRegressor()
# Búsqueda aleatoria usando 3-fold CV, 
# Busca sobre 100 iteraciones 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Ajusta sobre los datos
rf_random.fit(X, Y)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  8.0min finished


{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}

Ahra vamos a ver si la búsqueda aleatoria nos da un mejor modelo que el modelo base.

In [24]:
def evaluate(model, test_features, test_labels, name):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Performance of {:s}'.format(name))
    print('Error promedio: {:0.4f}'.format(np.mean(errors)))
    print('Precisión = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X, Y)
base_accuracy = evaluate(base_model, X, Y, 'Base')

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X, Y, 'Mejor Aleatorio')

print('Mejora de {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Performance of Base
Error promedio: 0.0412
Precisión = 99.66%.
Performance of Mejor Aleatorio
Error promedio: 0.0184
Precisión = 99.85%.
Mejora de 0.19%.


Mejoramos, marginalmente, pero mejoramos. También hay que tomar en cuenta que ambas evaluaciones están hechas sobre los mismos datos que se usaron para entrenar los modelos!!!!

Ahora terminemos el proceso de selección de hiperparámetros con una búsqueda exahustiva sobre una malla reducida. Esta malla sólo se mueve un poco alrededor de los parámetros seleccionados en el método aleatorio

In [27]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [70, 80, 90, 100],
    'max_features': [11, 13, 15],
    'min_samples_leaf': [1, 3, 5],
    'min_samples_split': [3, 5, 10],
    'n_estimators': [600, 700, 800, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X, Y)
grid_search.best_params_

Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   49.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1296 out of 1296 | elapsed:  7.0min finished


{'bootstrap': False,
 'max_depth': 100,
 'max_features': 15,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 600}

Teóricamente esta debería ser una refinación de lo que ya obtuvimos. Probémoslo

In [29]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X, Y, 'Grid')

print('Mejora de {:0.2f}%.'.format( 100 * (grid_accuracy - random_accuracy) / random_accuracy))

Performance of Grid
Error promedio: 0.0069
Precisión = 99.94%.
Mejora de 0.10%.
